In [70]:
import numpy as np
import pandas as pd
import os
from celltypist import models
from tqdm import tqdm

In [71]:
import celltypist
celltypist.models.models_description()


👉 Detailed model information can be found at `https://www.celltypist.org/models`


,model,description
0,Immune_All_Low.pkl,immune sub-populations combined from 20 tissue...
1,Immune_All_High.pkl,immune populations combined from 20 tissues of...
2,Adult_COVID19_PBMC.pkl,peripheral blood mononuclear cell types from C...
3,Adult_CynomolgusMacaque_Hippocampus.pkl,cell types from the hippocampus of adult cynom...
4,Adult_Human_MTG.pkl,cell types and subtypes (10x-based) from the a...
5,Adult_Human_PancreaticIslet.pkl,cell types from pancreatic islets of healthy a...
6,Adult_Human_PrefrontalCortex.pkl,cell types and subtypes from the adult human d...
7,Adult_Human_Skin.pkl,cell types from human healthy adult skin
8,Adult_Human_Vascular.pkl,vascular populations combined from multiple ad...
9,Adult_Mouse_Gut.pkl,cell types in the adult mouse gut combined fro...


In [61]:
DATA_NAME = 'PBMC10k_PBMC'
MODEL_NAME = 'Healthy_COVID19_PBMC'       # какая модель CellTypist
TOP_N_GENES_PER_CLASS = 300         # сколько генов на класс
OUT_GENES_LIST = f'../../Datasets/{DATA_NAME}/output/celltypist/celltypist_core_genes.txt'
OUT_TABLE = f'../../Datasets/{DATA_NAME}/output/celltypist/celltypist_core_genes_per_class.csv'

In [62]:
model_dir = os.path.expanduser("~/.celltypist/data/models")
model_path = os.path.join(model_dir, MODEL_NAME + ".pkl")
print(f'Загружаю модель из: {model_path}')

model = models.Model.load(model_path)
print(model)

Загружаю модель из: /home/salimovdr/.celltypist/data/models/Healthy_COVID19_PBMC.pkl
CellTypist model with 51 cell types and 3443 features
    date: 2022-03-10 05:08:08.224597
    details: peripheral blood mononuclear cell types from healthy and COVID-19 individuals
    source: https://doi.org/10.1038/s41591-021-01329-2
    version: v1
    cell types: ASDC, B_exhausted, ..., pDC
    features: HES4, ISG15, ..., MT-CYB


In [63]:
clf = model.classifier

weights = clf.coef_                 # shape: (n_classes, n_genes)
genes = clf.features       # массив имен генов
celltypes = clf.classes_            # массив названий классов

print(f'Модель содержит {len(celltypes)} классов и {len(genes)} генов.')

Модель содержит 51 классов и 3443 генов.


In [64]:
top_genes_per_class = []
rows = []

for i, ct in tqdm(enumerate(celltypes)):
    class_weights = weights[i, :]                     # веса для данного класса
    # сортировка по |weight| по убыванию
    idx_sorted = np.argsort(np.abs(class_weights))[::-1]
    idx_top = idx_sorted[:TOP_N_GENES_PER_CLASS]
    genes_top = genes[idx_top]
    weights_top = class_weights[idx_top]

    top_genes_per_class.append(set(genes_top))

    # копим табличку «класс–ген–вес»
    for g, w in zip(genes_top, weights_top):
        rows.append({
            'cell_type': ct,
            'gene': g,
            'weight': w,
            'abs_weight': abs(w),
        })

51it [00:00, 3386.52it/s]


In [65]:
# 5. Объединяем всё в ядро
core_genes = sorted(set().union(*top_genes_per_class))
print(f'Всего уникальных генов в ядре: {len(core_genes)}')

Всего уникальных генов в ядре: 2424


In [66]:
# 6. Сохранение: просто список генов
with open(OUT_GENES_LIST, 'w') as f:
    for g in core_genes:
        f.write(g + '\n')
print(f'Ядро генов сохранено в {OUT_GENES_LIST}')

# 7. Сохранение: подробная таблица
df = pd.DataFrame(rows)
df.sort_values(['cell_type', 'abs_weight'], ascending=[True, False], inplace=True)
df.to_csv(OUT_TABLE, index=False)
print(f'Tаблица ядра генов по классам сохранена в {OUT_TABLE}')

Ядро генов сохранено в ../../Datasets/PBMC10k_PBMC/output/celltypist/celltypist_core_genes.txt
Tаблица ядра генов по классам сохранена в ../../Datasets/PBMC10k_PBMC/output/celltypist/celltypist_core_genes_per_class.csv
